In [1]:
# don't need to look, this exploits smooth primes :/

In [8]:
from Crypto.Util.number import getPrime
from sympy import isprime

def get_smooth_prime(bits, smoothness=16):
    p = 2
    p_factors = [p]
    while p.bit_length() < bits - 2 * smoothness:
        factor = getPrime(smoothness)
        p_factors.append(factor)
        p *= factor
    bitcnt = (bits - p.bit_length()) // 2
    while True:
        prime1 = getPrime(bitcnt)
        prime2 = getPrime(bitcnt)
        tmpp = p * prime1 * prime2
        if tmpp.bit_length() < bits:
            bitcnt += 1
            continue
        if tmpp.bit_length() > bits:
            bitcnt -= 1
            continue
        if isprime(tmpp + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = tmpp + 1
            break
    p_factors.sort()
    return (p, p_factors)
sprime, sprime_factors = get_smooth_prime(1536)

In [ ]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse
from typing import List

Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return str(p.x)+","+str(p.y) if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "a b p")

def elliptic_curve_addition(curve: EllipticCurve, Q: Point, R: Point):
    assert not (Q.isinf and R.isinf), "cannot add O to O"
    if Q.isinf: return R
    if R.isinf: return Q
    if (Q.x==R.x and Q.y==-R.y): return Point(0,0,isinf=True)
    tangent = (R.y - Q.y)*inverse(R.x - Q.x, curve.p) if Q!=R else (3*(Q.x)**2 + curve.a)*inverse(2*Q.y, curve.p)
    
    x = tangent**2 - Q.x - R.x
    y = tangent*(Q.x - x) - Q.y
    return Point(x % curve.p, y % curve.p)

def elliptic_curve_multiplication(curve: EllipticCurve, p: Point, n: int):
    Q = p
    R = Point(0,0,True)
    while (n>0):
        if n%2==1: R = elliptic_curve_addition(curve, Q, R)
        Q = elliptic_curve_addition(curve, Q, Q)
        n = n//2
    return R

def elliptic_curve_inverse(curve: EllipticCurve, p: Point):
    return Point(p.x, - p.y % curve.p)

def elliptic_curve_crt(curve: EllipticCurve, C: List[Point], N: List[int]):
    total = Point(0,0)
    modulo = prod(N)
    for n_i, c_i in zip(N, C):
        p = modulo // n_i
        total = elliptic_curve_addition(curve, total, elliptic_curve_multiplication(curve, c_i, inverse(p, n_i) * p))
    return total

curve = EllipticCurve(310717010502520989590157367261876774703, 2, 3)
elliptic_curve_crt(curve, [Point(2,2), Point(13, 37), Point(0, 0)], [2, 137, 68327521]) # , Point(13, 37), Point(13, 37) , 847685303, 19578713402441478521

In [ ]:
from collections import Counter
def _discrete_log_pohlig_hellman(n, a, b, order, order_factors):
    l = [0] * len(order_factors)
    for i, (pi, ri) in enumerate(order_factors.items()):
        for j in range(ri):
            aj = elliptic_curve_multiplication(curve, elliptic_curve_addition(a, elliptic_curve_multiplication(curve, b, -l[i])), order // pi**(j + 1))
            bj = elliptic_curve_multiplication(curve, b, order // pi)
            cj = smooth_discrete_log(n, aj, bj, pi, True)
            l[i] += elliptic_curve_multiplication(cj, pi**j)

    d, _ = elliptic_curve_crt([pi**ri for pi, ri in order_factors.items()], l)
    return d

def smooth_discrete_log(n, a: Point, b: Point, sp_factors: list):
    factors = dict(Counter(sp_factors).most_common())
    order = 1
    for px, kx in factors.items():
        order *= px**kx
    order_factors = {}
    for p, e in factors.items():
        i = 0
        for _ in range(e):
            if pow(b, order // p, n) == 1:
                order //= p
                i += 1
            else:
                break
        if i < e:
            order_factors[p] = e - i
    return _discrete_log_pohlig_hellman(n, a, b, order, order_factors)

smooth_discrete_log

In [ ]:


p = 310717010502520989590157367261876774703
a = 2
b = 3

g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = Point(g_x, g_y)

smooth_discrete_log(p, a, 2, {2: 1, 137: 1, 68327521: 1, 847685303: 1, 19578713402441478521: 1})

# from Crypto.Hash import SHA1
# SHA1.new(str(flag.x).encode()).hexdigest()

In [6]:
from sympy import factorint
factorint(p-1)

{2: 1, 137: 1, 68327521: 1, mpz(847685303): 1, 19578713402441478521: 1}

In [ ]:
from collections import namedtuple
from math import inf
from Crypto.Util.number import inverse
Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return str(p.x)+","+str(p.y) if not p.isinf else "O"
Point.__str__ = pt_to_str
def elliptic_curve_addition(p1: Point, p2: Point):
    assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, p) if p1!=p2 else (3*(p1.x)**2 + a)*inverse(2*p1.y, p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%p, p3y%p)

def elliptic_curve_multiplication(p: Point, n: int):
    p1 = p
    p2 = Point(0,0,True)
    while (n>0):
        if n%2==1: p2 = elliptic_curve_addition(p1, p2)
        p1 = elliptic_curve_addition(p1, p1)
        n = n//2
    return p2

p = 310717010502520989590157367261876774703
a = 2
b = 3

g_x = 179210853392303317793440285562762725654
g_y = 105268671499942631758568591033409611165
G = Point(g_x, g_y)

smooth_discrete_log(p, a, 2, {2: 1, 137: 1, 68327521: 1, 847685303: 1, 19578713402441478521: 1})

# from Crypto.Hash import SHA1
# SHA1.new(str(flag.x).encode()).hexdigest()from sympy.utilities.misc import as_int
from collections import Counter
from sympy import isprime
def _discrete_log_pohlig_hellman(n, a, b, order, order_factors):
    from .modular import crt
    a %= n
    b %= n

    l = [0] * len(order_factors)

    for i, (pi, ri) in enumerate(order_factors.items()):
        for j in range(ri):
            aj = pow(a * pow(b, -l[i], n), order // pi**(j + 1), n)
            bj = pow(b, order // pi, n)
            cj = smooth_discrete_log(n, aj, bj, pi, True)
            l[i] += cj * pi**j

    d, _ = crt([pi**ri for pi, ri in order_factors.items()], l)
    return d

def smooth_discrete_log(n, a, b, sp_factors: list):
    factors = dict(Counter(sp_factors).most_common())
    order = 1
    for px, kx in factors.items():
        order *= px**kx
    order_factors = {}
    for p, e in factors.items():
        i = 0
        for _ in range(e):
            if pow(b, order // p, n) == 1:
                order //= p
                i += 1
            else:
                break
        if i < e:
            order_factors[p] = e - i
    return _discrete_log_pohlig_hellman(n, a, b, order, order_factors)

In [ ]:
out = pow(2, 65537, sprime)
smooth_discrete_log(sprime, out, 2, sprime_factors)

In [21]:
# actual start here, above was trying to write pohlin hellman for elliptic curves

In [23]:
def smooth_discrete_log(n: int, a: int, b: int, sp_factors: list, order=None, prime_order=None):
    if order is None:
        factors = dict(Counter(sp_factors).most_common())
        order = 1
        for px, kx in factors.items():
            order *= px**kx
        order_factors = {}
        for p, e in factors.items():
            i = 0
            for _ in range(e):
                if pow(b, order // p, n) == 1:
                   order //= p
                   i += 1
                else:
                    break
            if i < e:
                order_factors[p] = e - i
    if prime_order is None:
        prime_order = isprime(order)
    return _discrete_log_pohlig_hellman(n, a, b, order, order_factors)
smooth_discrete_log(310717010502520989590157367261876774703, out, 2, {2: 1, 137: 1, 68327521: 1, mpz(847685303): 1, 19578713402441478521: 1})

{2: 1, 137: 1, 68327521: 1, mpz(847685303): 1, 19578713402441478521: 1}